<a href="https://colab.research.google.com/github/stmeinert/Recolorization_IANN/blob/main/preprocess_using_colab_and_zip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
print(os.getcwd())
print(os.listdir(os.getcwd()))

/content
['.config', 'drive', 'sample_data']


In [2]:
# need to unzip my files
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/right_celeb_upload/img_align_celeba_zipped.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/unzip_celeb')

In [3]:
import pathlib
#directory of the image data for preprocessing
data_dir = pathlib.Path('/content/unzip_celeb/img_align_celeba')
image_count = len(list(data_dir.glob('*.jpg')))
print('Number of images in directory' + str(image_count))

Number of images in directory202599


In [4]:
!pip install tensorflow-io

     |████████████████████████████████| 23.4 MB 1.2 MB/s 


In [5]:
path = os.path.join(os.getcwd(), "celeb_data_set_preprocessed_tiny_64_30000")
print('Path of the saved dataset file' + str(path))

Path of the saved dataset file/content/celeb_data_set_preprocessed_tiny_64_30000


In [6]:
import numpy as np
import os
import tensorflow as tf
import pathlib
import tensorflow_io as tfio


SIZE = (128,128)
BATCH_SIZE = None # changed to just one

#################################################
# Prepare data
#################################################

def resize(image):
    return tf.image.resize_with_pad(image, target_height=SIZE[0], target_width=SIZE[1], method=tf.image.ResizeMethod.BILINEAR)


def to_lab(image):
    # expects input to be normalized to [0;1]!!
    # output channels are [l,a,b]
    return tfio.experimental.color.rgb_to_lab(image)


def to_grayscale(image):
    # take l channel (size index starts at one^^)
    image = tf.slice(image, begin=[0, 0, 0], size=[-1, -1, 1])
    return image

def prepare_image_data_just_pictures(image_ds):
    # resize image to desired dimension, replace label with colored image
    image_ds = image_ds.map(lambda x: (resize(x), resize(x)))

    # normalize data to [0;1) for lab encoder
    image_ds = image_ds.map(lambda image, target: ((image/256), (target/256)))

    # convert image and target image to lab color space
    image_ds = image_ds.map(lambda image, target: (to_lab(image), to_lab(target)))

    # only take l channel of input tensor
    image_ds = image_ds.map(lambda image, target: (to_grayscale(image), target))

    # l in lab is in [0;100] -> normalize to [-1;1]
    # ab are in range [-128;127]
    image_ds = image_ds.map(lambda image, target: ((image/50)-1, target))

    image_ds = image_ds.shuffle(1000).prefetch(20)
    return image_ds




#directory of the image data for preprocessing
data_dir = pathlib.Path('/content/unzip_celeb/img_align_celeba')
image_count = len(list(data_dir.glob('*.jpg')))
print('Number of images in directory' + str(image_count))


#create the dataset 
#inital size
img_height = 218
img_width = 178

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  labels=None,
  #changed from 0.7 to 0.85
  validation_split=(0.85),
  subset='training',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=None)
print('created data set')


#apply preprocessing step

train_ds = train_ds.apply(prepare_image_data_just_pictures)
print('applied preprocessing')




# save the dataset
# folder of the preprocessed data
path = os.path.join(os.getcwd(), "celeb_data_set_unbatch_30000")
# Save a dataset
tf.data.experimental.save(train_ds, path,compression= 'GZIP')
print('Path of the saved dataset file' + str(path))

Number of images in directory202599
Found 202599 files belonging to 1 classes.
Using 30390 files for training.
created data set
applied preprocessing
Path of the saved dataset file/content/celeb_data_set_unbatch_30000


In [8]:
!zip -r /content/celeb_data_set_unbatch_30000.zip  /content/celeb_data_set_unbatch_30000

  adding: content/celeb_data_set_unbatch_30000/ (stored 0%)
  adding: content/celeb_data_set_unbatch_30000/snapshot.metadata (stored 0%)
  adding: content/celeb_data_set_unbatch_30000/dataset_spec.pb (deflated 41%)
  adding: content/celeb_data_set_unbatch_30000/2991326379668033694/ (stored 0%)
  adding: content/celeb_data_set_unbatch_30000/2991326379668033694/00000000.shard/ (stored 0%)
  adding: content/celeb_data_set_unbatch_30000/2991326379668033694/00000000.shard/00000000.snapshot (deflated 0%)


In [13]:
!mkdir /content/drive/MyDrive/celeb_data_set_unbatch_30000

mkdir: cannot create directory ‘/content/drive/MyDrive/celeb_data_set_unbatch_30000’: File exists


In [14]:
!cp /content/celeb_data_set_unbatch_30000.zip /content/drive/MyDrive/celeb_data_set_unbatch_30000/celeb_data_set_unbatch_30000.zip

In [15]:
!ls -lh /content/celeb_data_set_unbatch_30000.zip

-rw-r--r-- 1 root root 4.9G Mar 20 16:02 /content/celeb_data_set_unbatch_30000.zip
